In [1]:
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

from torch import nn, optim

In [14]:
# loading out plant disease detector class
from models import PlantDiseaseDetector

In [15]:
# Creating image transformers
transform = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop((2224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [16]:
# Loading the datasets
train_dir = "training_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "training_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

In [17]:
train_data = datasets.ImageFolder(train_dir, transform = transform)
valid_data = datasets.ImageFolder(valid_dir, transform = transform)

In [18]:
# defining the dataloaders
batch_size = 20
data_loaders = {
    'train': DataLoader(train_data, batch_size = batch_size, shuffle = True),
    'valid': DataLoader(valid_data, batch_size = batch_size, shuffle = True),
}

In [33]:
# Creating an instance of the PlantDiseaseDetector model
plant_model_v1 = PlantDiseaseDetector(num_classes=len(train_data.classes))

In [34]:
print(plant_model_v1)

PlantDiseaseDetector(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

## Training the Model

In [35]:
#setting the loss function
loss_fn = nn.CrossEntropyLoss()

# setting the optimizer
optimizer = optim.Adam(plant_model_v1.parameters(), lr=0.001)

In [36]:
# setting the device for device-agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [37]:
# putting the model to device
plant_model_v1.to(device)

PlantDiseaseDetector(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

In [38]:
# Training loop
epochs = 20

for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0

    train_correct = 0
    train_total = 0
    valid_correct = 0
    valid_total = 0

    plant_model_v1.train()
    # getting data from generators
    for inputs, labels in data_loaders['train']:
        # putting data to device
        inputs, labels = inputs.to(device), labels.to(device)
        labels_preds = plant_model_v1(inputs)

        loss = loss_fn(labels_preds, labels)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        train_loss  += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(labels_preds.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    plant_model_v1.eval()
    with torch.inference_mode():
        for inputs, labels in data_loaders['valid']:
            inputs, labels = inputs.to(device), labels.to(device)
            labels_preds_valid = plant_model_v1(inputs)
            loss_valid = loss_fn(labels_preds_valid, labels)
            valid_loss += loss_valid.item()

            # Calculate validation accuracy
            _, predicted = torch.max(labels_preds_valid.data, 1)
            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()


    # Calculate and print accuracies
    train_accuracy = 100 * train_correct / train_total
    valid_accuracy = 100 * valid_correct / valid_total

    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {train_loss/len(data_loaders['train']):.3f}.. "
          f"Validation loss: {valid_loss/len(data_loaders['valid']):.3f}.. "
          f"Train Accuracy: {train_accuracy:.3f}%.. "
          f"Validation Accuracy: {valid_accuracy:.3f}%")


Epoch 1/20.. Train loss: 2.529.. Validation loss: 2.040.. Train Accuracy: 35.065%.. Validation Accuracy: 42.579%
Epoch 2/20.. Train loss: 1.555.. Validation loss: 1.323.. Train Accuracy: 60.100%.. Validation Accuracy: 64.967%
Epoch 3/20.. Train loss: 1.218.. Validation loss: 1.190.. Train Accuracy: 67.914%.. Validation Accuracy: 65.695%
Epoch 4/20.. Train loss: 1.042.. Validation loss: 1.197.. Train Accuracy: 71.928%.. Validation Accuracy: 65.456%
Epoch 5/20.. Train loss: 0.924.. Validation loss: 1.327.. Train Accuracy: 74.792%.. Validation Accuracy: 59.714%
Epoch 6/20.. Train loss: 0.845.. Validation loss: 0.914.. Train Accuracy: 76.358%.. Validation Accuracy: 72.940%
Epoch 7/20.. Train loss: 0.781.. Validation loss: 0.843.. Train Accuracy: 78.119%.. Validation Accuracy: 75.256%
Epoch 8/20.. Train loss: 0.729.. Validation loss: 0.863.. Train Accuracy: 79.401%.. Validation Accuracy: 72.695%
Epoch 9/20.. Train loss: 0.694.. Validation loss: 0.876.. Train Accuracy: 80.037%.. Validation A

In [39]:
torch.save(plant_model_v1.state_dict(), "trained_models/plant_model_v1.pth")